In [33]:
import pandas as pd 
preop_imaging_df = pd.read_excel('../Data/side_concordance_results.xlsx')


In [34]:
def extract_side(location):
    if pd.isnull(location):
        return 'No Scan'
    elif 'Left' in location:
        return 'Left'
    elif 'Right' in location:
        return 'Right'
    else:
        return 'Unknown'

In [35]:
# Define modalities and concordance column
spect_modality = 'SPECT/CT'
modality_4d = '4D CT Scan'
concord_col_4d = modality_4d + '_Concordance'

# Filter for SPECT negative (non-localizing)
negative_spect = preop_imaging_df[preop_imaging_df[spect_modality] == 'Non-localizing'].copy()

# Apply extract_side to 4D CT Scan results within the negative_spect dataframe
negative_spect['4D CT Imaging Side'] = negative_spect[modality_4d].apply(extract_side)

# Filter for rows where 4D CT localized to Left or Right (i.e. positive scan)
spect_neg_4d_pos = negative_spect[
    negative_spect['4D CT Imaging Side'].isin(['Left', 'Right'])
].copy()


In [36]:
# Calculate total number of SPECT- / 4D CT+ cases
total_cases = spect_neg_4d_pos.shape[0]

# Count exact matches
exact_count = spect_neg_4d_pos[spect_neg_4d_pos[concord_col_4d] == 'Yes (Exact)'].shape[0]

# Count side-only matches
side_count = spect_neg_4d_pos[spect_neg_4d_pos[concord_col_4d] == 'Yes (Side)'].shape[0]

# Count incorrect localizations (i.e. any positive that wasn't exact or side match)
incorrect_count = total_cases - (exact_count + side_count)

# Calculate percentages
exact_pct = (exact_count / total_cases) * 100 if total_cases > 0 else None
side_pct = (side_count / total_cases) * 100 if total_cases > 0 else None
incorrect_pct = (incorrect_count / total_cases) * 100 if total_cases > 0 else None

In [ ]:
# Create final results dataframe
spect_neg_4d_pos_summary_df = pd.DataFrame({
    'Match Type': ['Exact Match', 'Side-only Match', 'Incorrect Localization'],
    'Count': [exact_count, side_count, incorrect_count],
    'Percentage': [
        f"{exact_pct:.2f}%" if exact_pct is not None else 'N/A',
        f"{side_pct:.2f}%" if side_pct is not None else 'N/A',
        f"{incorrect_pct:.2f}%" if incorrect_pct is not None else 'N/A'
    ]
})

# Display results
print(f"Total SPECT negative / 4D CT positive cases: {total_cases}")
display(spect_neg_4d_pos_summary_df)

Total SPECT negative / 4D CT positive cases: 31


,Match Type,Count,Percentage
0,Exact Match,13,41.94%
1,Side-only Match,6,19.35%
2,Incorrect Localization,12,38.71%
